#### Documentation
**Before using this notebook, please click the *cells hidden* button below to show the documentation.**
##### License
> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.
##### About this software
> This notebook processes a **[CHARMM-GUI](https://www.charmm-gui.org/) system archive** (`.tgz`), producing a **GROMACS-ready folder for production runs**.
>
> A protein system prepared with the CHARMM-GUI **Solution Builder** or **Membrane Builder** must be provided.
>
> <font color="maroon">Warning: In CHARMM-GUI, the option to generate GROMACS compatible outputs **must** be enabled.</font>
>
> The recommended **minimisation** and **equilibration** simulations are then run with **GROMACS**, which automatically utilises the GPU if one is allocated. The equilibrated system is saved for a later production simulation.
##### Installation
> The installation notebook, [`Build_to_Google_Drive.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/Build_to_Google_Drive.ipynb), must be run before using this notebook.
##### Piecewise preparation of protein-ligand complexes
> *Optionally*, a docked ligand conformation prepared with the CHARMM-GUI **Ligand Reader** may be provided, in which case the two separate structures and topologies will be **merged into a protein-ligand complex**.
>
> <font color="maroon">Warning: The protein `.tgz` **must** be the **protein docked against**, and the ligand `.tgz` **must** be the **docking output**.</font>
> To merge **multiple** cooperatively bound ligands, **multiple paths to archive files** are separated by the `|` keyword.
> *e.g.*
> ```
> ligand_archives = "{GoogleDrive}/diazepam.tgz | {GoogleDrive}/GABA.tgz"
> ```
> The above example merges docked conformations of diazepam and GABA into a protein system.

In [ ]:
import os
import re
import shutil
from functools import partial

#@markdown Provide the location of the `.tgz` from **Solution Builder** or **Membrane Builder**.
protein_archive = "{GoogleDrive}/CHARMM-GUI/7FBF_FABPH.tgz" #@param {type: "string"}

#@markdown Specify a new folder in which to save the equilibrated output -- the production simulation can then be run in this folder.
output_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}

#@markdown If applicable, please see also the advanced settings below. **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**

#@markdown \
#@markdown **Merging docked ligands into the system**
#@markdown Optionally, docked ligand conformations may be added to the system, provided as `.tgz` archives from **Ligand Reader**.
ligand_archives = "{GoogleDrive}/CHARMM-GUI/7FBF_octanoic_acid.tgz" #@param {type: "string"}

#@markdown \
#@markdown **Expert settings**
#@markdown Optionally, equilibrate the system for double the suggested time.
double_equilibration_length = False #@param {type: "boolean"}

#@markdown Optionally, remove correction maps (CMAP terms). This will **[decrease the accuracy](https://pubs.acs.org/doi/10.1021/ct900549r)** of the production simulation but substantially improve performance on high-end GPUs (e.g. A100).
remove_cmap_terms = False #@param {type: "boolean"}

# Google Drive
if not os.path.isdir("/content/drive/MyDrive"):
  from google.colab import drive
  drive.mount("/content/drive")
if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")

# Methods for parsing and validation
def _multiple(s):
  return list(filter(None, s.split("|")))
def _path(s, exists=False, exts=None):
  if "{GoogleDrive}" in s and not s.startswith("{GoogleDrive}"): raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {s}")
  s = s.format(GoogleDrive="/content/drive/MyDrive")
  if exists and not os.path.isfile(s): raise FileNotFoundError(f"Error: file not found: {s}")
  if exts and not any(s.endswith(ext) for ext in exts): raise ValueError(f"Error: expecting file extension like {', '.join(exts)}, but got: {s}")
  return os.path.abspath(s)
_extra_spaces = re.compile(r"(^ +|(?<=\|) +| +(?=\|)| +$)")
def parse(s, multiple=False, mandatory=False, path=False, exists=False, exts=None):
  s = _extra_spaces.sub("", s)
  if multiple:
    s = _multiple(s)
  if mandatory and (multiple and not any(v for v in s) or not multiple and not s):
    raise ValueError("Error: mandatory setting without value")
  if path:
    _path_preset = partial(_path, exists=exists, exts=exts)
    if multiple: s = [_path_preset(v) for v in s]
    else: s = _path_preset(s)
  return s

archive_exts = [".tgz", ".tar.gz"]
protein_archive = parse(protein_archive, mandatory=True, path=True, exists=True, exts=archive_exts)
output_folder = parse(output_folder, mandatory=True, path=True)
os.makedirs(output_folder, exist_ok=True)
if os.path.isfile(output_folder + "/conf.gro"):
  raise RuntimeError(f"Error: expecting empty folder, but found existing output: {output_folder}")

ligand_archives = parse(ligand_archives, multiple=True, path=True, exists=True, exts=archive_exts)
ligands_bash = "|".join(ligand_archives)

do_double_eq_bash = "true" if double_equilibration_length else "false"
remove_cmaps_bash = "true" if remove_cmap_terms else "false"

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}

try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
os.makedirs("scratch")
%cd "scratch"

In [ ]:
%%bash -s "$protein_archive"
protein_archive="$1"

#@markdown Extract the system from the protein archive. This should come from either **Solution Builder** or **Membrane Builder**.
if [[ -d "protein" ]]; then
  exit 0 # already extracted protein
fi
if [[ ! -s "${protein_archive}" ]]; then
  echo "Error: file not found: ${protein_archive}" >&2
  exit 1
fi
tar -xzf "${protein_archive}"
if ! compgen -G "charmm-gui-*/" > /dev/null; then
  echo "Error: not in CHARMM-GUI archive format: ${protein_archive}" 1>&2
  exit 1
fi
mv charmm-gui-*/ "protein"

In [ ]:
%%bash -s "$ligands_bash"
ligand_archives="$1"

#@markdown Extract the docked ligands from the ligand archives, if given. These files **must** come from putting the **docking output** into **Ligand Reader**.
if compgen -G "ligand_*/" > /dev/null; then
  exit 0 # already extracted ligand(s)
fi
i=1
IFS="|"
for ligand_archive in $ligand_archives; do
  if [[ ! -s "${ligand_archive}" ]]; then
    echo "Error: file not found: ${ligand_archive}" >&2
    exit 1
  fi
  tar -xzf "${ligand_archive}"
  if ! compgen -G "charmm-gui-*/" > /dev/null; then
    echo "Error: not in CHARMM-GUI archive format: ${ligand_archive}" 1>&2
    exit 1
  fi
  mv charmm-gui-*/ "ligand_${i}"
  i=$(($i + 1))
done

In [ ]:
#@markdown In the following cells, applications are downloaded from a **persistent cache** in your Google Drive.
#@markdown This cell sets up the cache folder.
storage = "/content/drive/MyDrive/gromacs-on-colab"
%env STORAGE={storage}

In [ ]:
%%bash
#@markdown Install Miniconda environments for `cgenff_charmm2gmx.py` and **Biopython** / **Open Babel** from cache.
if [[ -d "${START}/miniconda3" ]]; then
  exit 0 # already installed
fi
miniconda3_vers="py310_23.5.2-0" #@param {type: "string"}
cache_miniconda3_installer="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64.sh.tar.gz"
if [[ -s "${cache_miniconda3_installer}" ]]; then
  tar -xzf "${cache_miniconda3_installer}"
else
  echo "Error: Miniconda installer not found" >&2
  echo "(Have you installed Miniconda to your Google Drive?)" >&2
  exit 1
fi
bash "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" -b -p "${START}/miniconda3"
rm "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"
eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
cache_miniconda3="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64_envs.tar.gz"
if [[ -s "${cache_miniconda3}" ]]; then
  tar -xzf "${cache_miniconda3}" -C "${START}/miniconda3"
else
  echo "Error: Miniconda environments not found" >&2
  echo "(Have you installed Miniconda to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%bash
#@markdown Install Miniconda environments for `cgenff_charmm2gmx.py` and **Biopython** / **Open Babel** from cache.
if [[ -d "${START}/miniconda3" ]]; then
  exit 0 # already installed
fi
miniconda3_vers="py310_23.5.2-0" #@param {type: "string"}
cache_miniconda3_installer="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64.sh.tar.gz"
if [[ -s "${cache_miniconda3_installer}" ]]; then
  tar -xzf "${cache_miniconda3_installer}"
else
  echo "Error: Miniconda installer not found" >&2
  echo "(Have you installed Miniconda to your Google Drive?)" >&2
  exit 1
fi
bash "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" -b -p "${START}/miniconda3"
rm "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"
eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
cache_miniconda3="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64_envs.tar.gz"
if [[ -s "${cache_miniconda3}" ]]; then
  tar -xzf "${cache_miniconda3}" -C "${START}/miniconda3"
else
  echo "Error: Miniconda environments not found" >&2
  echo "(Have you installed Miniconda to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%bash
#@markdown The CHARMM36 forcefield is downloaded from cache.
if [[ -d "${START}/charmm36.ff" ]]; then
  exit 0 # already installed
fi
charmm36_vers="jul2022" #@param {type: "string"}
cache_charmm36="${STORAGE}/charmm36-${charmm36_vers}.tar.gz"
if [[ -s "${cache_charmm36}" ]]; then
  tar -xzf "${cache_charmm36}" -C "${START}"
else
  echo "Error: CHARMM36 forcefield installation not found" >&2
  echo "(Have you installed the CHARMM36 forcefield to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%bash
#@markdown The utility **`cgenff_charmm2gmx.py`** is installed from cache.
if [[ -x "$START/miniconda3/envs/charmm2gmx/bin/cgenff_charmm2gmx.py" ]]; then
  exit 0 # already installed
fi
charmm2gmx_vers="py3_nx2" #@param {type: "string"}
cache_charmm2gmx="$STORAGE/cgenff_charmm2gmx_${charmm2gmx_vers}.tar.gz"
if [[ -s "${cache_charmm2gmx}" ]]; then
  tar -xzf "${cache_charmm2gmx}" -C "${START}/miniconda3/envs/charmm2gmx/bin"
else
  echo "Error: charmm2gmx installation not found" >&2
  echo "(Have you installed charmm2gmx to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
#@markdown A class which allows for the parsing and limited editing of **GROMACS ".itp" topology files**.
from collections import namedtuple
T = namedtuple("T", ["line", "comment"])
class Itp:
  def __init__(self, filename):
    self.blocks = [None]
    self.data = [[]]
    with open(filename) as f:
      self.__parse(f)
  def __parse(self, lines):
    columns = re.compile(r"\b(?=[ \t])")
    concat = None
    for l in lines:
      l = l.rstrip("\n")
      l = l.replace("\t", " ")
      if l.endswith("\\"):
        if concat is None:
          concat = l[:-1] + " "
        else:
          concat += l[:-1] + " "
        continue
      elif concat is not None:
        l = concat + l
        concat = None
      try:
        l, c = l.split(";")
      except ValueError:
        c = None
      l_ = l.strip()
      if l_.startswith("["):
        self.blocks.append(T(line=l, comment=c))
        self.data.append([])
      elif l_ and not l_.startswith("#"):
        s = columns.split(l)
        self.data[-1].append(T(line=s, comment=c))
      else:
        self.data[-1].append(T(line=l, comment=c))
  def print(self, file=None):
    for block, data in zip(self.blocks, self.data):
      if block is not None:
        print(self._str(block), file=file)
      for t in data:
        print(self._str(t), file=file)
  def _str(self, t):
    l = t.line if isinstance(t.line, str) else "".join(t.line)
    if t.comment is not None:
      l += ";" + t.comment
    return l
  @classmethod
  def only(cls, data):
    return filter(lambda t: isinstance(t[0], list), data)

  directive_order_hint = [None, "defaults", "atomtypes", "bondtypes", "pairtypes", "angletypes", "dihedraltypes", "constrainttypes", "nonbond_params"]
  block_num_atom_cols = {
    "atoms": 1, "bonds": 2, "pairs": 2,
    "pairs_nb": 2, "angles": 3, "dihedrals": 4,
    "exclusions": -1, "constraints": 2, "settles": 1,
    "virtual_sites1": 2, "virtual_sites2": 3, "virtual_sites3": 4,
    "virtual_sites4": 5, "virtual_sitesn": 1, "position_restraints": 1,
    "distance_restraints": 2, "dihedral_restraints": 4, "orientation_restraints": 2,
    "angle_restraints": 4, "angle_restraints_z": 2
  }

In [ ]:
#@markdown A function which modifies an Itp topology object `x`, changing the order of its atom definitions such that hydrogens immediately follow their bonded heavy atoms.
from types import SimpleNamespace
S = lambda: SimpleNamespace(atom_obj=dict(), atom_element=dict(), atom_bonded_atoms=dict())
def interleave_H_in_topology(x):
  element = re.compile(r"(?<=[a-zA-Z])(?=[0-9])")
  cache = dict()
  name = None
  for block, data in zip(x.blocks, x.data):
    if block is None: continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
      cache[name] = S()
    elif "[atoms]" in block_line_:
      for t in Itp.only(data):
        l, c = t
        atom_id = int(l[0])
        cache[name].atom_obj[atom_id] = t
        cache[name].atom_element[atom_id] = element.split(l[4])[0].strip()
    elif "[bonds]" in block_line_:
      for l, c in Itp.only(data):
        u, v = int(l[0]), int(l[1])
        for u_, v_ in ((u, v), (v, u)):
          try:
            cache[name].atom_bonded_atoms[u_].append(v_)
          except KeyError:
            cache[name].atom_bonded_atoms[u_] = [v_]

  name = None
  new_data = dict()
  for i, (block, data) in enumerate(zip(x.blocks, x.data)):
    if block is None: continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
    elif "[atoms]" in block_line_:
      new_data[i] = list()
      skip = list()
      for t in data:
        l, c = t
        if isinstance(l, list):
          atom_id = int(l[0])
          if atom_id not in skip:
            if cache[name].atom_element[atom_id] != "H":
              new_data[i].append(t)
              if atom_id in cache[name].atom_bonded_atoms:
                for other_id in cache[name].atom_bonded_atoms[atom_id]:
                  if cache[name].atom_element[other_id] == "H":
                    new_data[i].append(cache[name].atom_obj[other_id])
                    skip.append(other_id)
            elif atom_id not in cache[name].atom_bonded_atoms:
              new_data[i].append(t)
        else:
          new_data[i].append(t)
  for key, value in new_data.items():
    x.data[key] = value

  name = None
  new_id = None
  atom_id_map = None
  def str_like(number, template):
    return f"%{len(template)}d" % (number,)
  for block, data in zip(x.blocks, x.data):
    if block is None: continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
      new_id = 1
      atom_id_map = dict()
    elif "[atoms]" in block_line_:
      for l, c in Itp.only(data):
        atom_id = int(l[0])
        l[0] = str_like(new_id, l[0])
        atom_id_map[atom_id] = new_id
        new_id += 1
    else:
      for key, num_cols in Itp.block_num_atom_cols.items():
        if f"[{key}]" in block_line_:
          for l, c in Itp.only(data):
            if num_cols < 0:
              num_cols = len(l)
            for i in range(num_cols):
              this_id = int(l[i])
              l[i] = str_like(atom_id_map[this_id], l[i])
          break
  return atom_id_map

In [ ]:
#@markdown A class which allows for the parsing and limited editing of **GROMACS ".gro" coordinates files**.
class Gro:
  def __init__(self, filename):
    self.title = None
    self.info = list()
    self.pos = list()
    self.vel = None
    self.width = None
    self.box = None
    self.box_explicit_diag = None
    self.box_width = None
    with open(filename) as f:
      self.__parse(f)
  def __parse(self, lines):
    n = None
    for i, l in enumerate(lines):
      l = l.rstrip("\n")
      if i == 0: self.title = l
      elif i == 1: n = int(l)
      elif 2 <= i < 2 + n:
        s = l[:20]
        d = l[20:].rstrip()
        if i == 2:
          num_dots = len([x for x in l if x == "."])
          if num_dots == 3: pass
          elif num_dots == 6: self.vel = list()
          else: raise RuntimeError(f"Error: expected 3 or 6 data columns, but got: {d}")
          if len(d) % num_dots != 0:
            raise RuntimeError(f"Error: expected consistent column width for {num_dots} columns, but got: {d}")
          self.width = len(d) // num_dots
        info = (int(s[:5]), s[5:10].strip(), s[10:15].strip(), int(s[15:20]))
        self.info.append(info)
        pos = tuple(float(d[x:x+self.width]) for x in range(0, 3 * self.width, self.width))
        self.pos.append(pos)
        if self.vel is not None:
          vel = tuple(float(d[x:x+self.width]) for x in range(3 * self.width, 6 * self.width, self.width))
          self.vel.append(vel)
      elif i == 2 + n:
        l = l.rstrip()
        num_dots = len([x for x in l if x == "."])
        if num_dots == 3: self.box_explicit_diag = False
        elif num_dots == 9: self.box_explicit_diag = True
        else: raise RuntimeError(f"Error: expected 3 or 9 box columns, but got: {l}")
        if len(l) % num_dots != 0:
          raise RuntimeError(f"Error: expected consistent column width for box vector, but got: {l}")
        self.box_width = len(l) // num_dots
        self.box = tuple(float(l[x:x+self.box_width]) for x in range(0, len(l), self.box_width))
      else:
        raise RuntimeError(f"Error: expected EOF, but got: {l}")
  def print(self, file=None):
    print(self.title, file=file)
    print("%5d" % (len(self.info),), file=file)
    w = self.width
    p = w - 5
    if self.vel is not None:
      v = w - 4
      fmt = "%5d%-5s%5s%5d" + f"%{w}.{p}f" * 3 + f"%{w}.{v}f" * 3
      def fit(x):
        return ((x - 1) % 99999) + 1
      for info, pos, vel in zip(self.info, self.pos, self.vel):
        print(fmt % (fit(info[0]), info[1], info[2], fit(info[3]), *pos, *vel), file=file)
    else:
      fmt = "%5d%-5s%5s%5d" + f"%{w}.{p}f" * 3
      for info, pos in zip(self.info, self.pos):
        print(fmt % (*info, *pos), file=file)
    w = self.box_width
    b = w - 5
    fmt = f"%{w}.{b}f" * (9 if self.box_explicit_diag else 3)
    print(fmt % self.box, file=file)

In [ ]:
#@markdown A function which modifies a Gro coordinates object `y`, changing the order of its atoms to match an interleaved Itp `x`, as per the provided `atom_id_map`.
def rearrange_coordinates(y, atom_id_map):
  try:
    data = list(zip(y.info, y.pos, y.vel))
  except TypeError:
    data = list(zip(y.info, y.pos))
  new_data = list()
  for old_id, new_id in atom_id_map.items():
    t = data[old_id - 1]
    t = ((*t[0][:3], new_id), *t[1:])
    new_data.append(t)
  new_data_sep = list(zip(*new_data))
  try:
    y.info, y.pos, y.vel = new_data_sep
  except ValueError:
    y.info, y.pos = new_data_sep

In [ ]:
#@markdown A function which combines multiple "forcefield.itp" / "molecule.prm" topology files, taking care with directive order.
def combine_itp_files(filenames_in, filename_out):
  all_parsed_files = [Itp(filename) for filename in filenames_in]
  def index(blocks, key):
    return next((i for i, t in enumerate(blocks) if t == key or hasattr(t, "line") and t.line.replace(" ", "") == f"[{key}]"), None)
  with open(filename_out, "w") as out:
    for key in Itp.directive_order_hint:
      for x in all_parsed_files:
        i = index(x.blocks, key)
        if i is not None:
          if x.blocks[i] is not None:
            print(x._str(x.blocks[i]), file=out)
          for t in x.data[i]:
            print(x._str(t), file=out)
          if key == "defaults":
            break
    for x in all_parsed_files:
      for block, data in zip(x.blocks, x.data):
        if block is not None and "".join(c for c in block.line if c not in "[ ]") not in Itp.directive_order_hint:
          print(x._str(block), file=out)
          for t in data:
            print(x._str(t), file=out)

In [ ]:
%%writefile superpose.py
#!/usr/bin/env python3
import sys
import numpy as np
from Bio.PDB import Superimposer, PDBParser
par = PDBParser()
target = par.get_structure("target", sys.argv[1])[0]
query = par.get_structure("query", sys.argv[2])[0]
target_atoms = [res["CA"] for ch in target for res in ch if "CA" in res]
query_atoms = [res["CA"] for ch in query for res in ch if "CA" in res]
sup = Superimposer()
sup.set_atoms(target_atoms, query_atoms)
print(f"RMSD: {sup.rms}", file=sys.stderr)
ro, tr = sup.rotran
print("Transformation:", file=sys.stderr)
print(ro, file=sys.stderr)
print(tr, file=sys.stderr)
with open("rms.txt", "w") as o:
  print(sup.rms, file=o)
np.save("ro.npy", ro, allow_pickle=False)
np.save("tr.npy", tr, allow_pickle=False)

In [ ]:
%%writefile insert_molecules_auto_topol
#!/usr/bin/env bash
topol="$1"
insert_mol="$2"
replace_mol="$3"
shift 3
if [[ ! -s "$topol" || -z "$insert_mol" || -z "$replace_mol" ]]; then
  exit 1
fi
gmx insert-molecules "$@" &> "insert-molecules.log"
ret=$?
if (( $ret != 0 )); then
  cat "insert-molecules.log" >&2
  exit $ret
fi
num_inserted="$(egrep -o "Added [0-9]+ molecules" "insert-molecules.log" | awk '{ print $2 }')"
if (( $num_inserted == 0 )); then
  echo "Error: could not insert molecule ${insert_mol}, no changes made to topology file ${topol}" >&2
  exit 1
fi
num_replaced="$(egrep -o "Replaced [0-9]+ residues" "insert-molecules.log" | awk '{ print $2 }')"
awk \
  -v molins="$insert_mol" \
  -v numins=$num_inserted \
  -v molrep="$replace_mol" \
  -v numrep=$num_replaced \
  '
  !x
  x {
    if ($1 == molins) {
      print $1, $2 + numins
      inserted=1
    }
    else if ($1 == molrep) {
      print $1, $2 - numrep
    }
    else {
      print $0
    }
  }
  $0 ~ /\[ *molecules *\]/ {
    x=1
  }
  END {
    if (!inserted) {
      print molins, numins
    }
  }
  ' \
  "${topol}" \
  > "${topol}.new" \
&& mv "${topol}.new" "${topol}"

In [ ]:
%%writefile forcefield_POT_CLA.top
[ atomtypes ]
     CLA 17 35.4500 -1.000 A 4.04468018036e-01 6.276000e-01
     POT 19 39.0983 1.000 A 3.14264522824e-01 3.640080e-01
[ nonbond_params ]
    CLA POT 1 3.63575766873e-01 4.77812800000e-01
    POT OC 1 3.13952708273e-01 4.27604800000e-01
[ pairtypes ]
    CP1 CLA 1 3.71504765465e-01 1.62045623205e-01
    CP1 POT 1 3.26403017859e-01 1.23410269913e-01
    CP2 CLA 1 3.71504765465e-01 1.62045623205e-01
    CP2 POT 1 3.26403017859e-01 1.23410269913e-01
    CP3 CLA 1 3.71504765465e-01 1.62045623205e-01
    CP3 POT 1 3.26403017859e-01 1.23410269913e-01
    CT1 CLA 1 3.71504765465e-01 1.62045623205e-01
    CT1 POT 1 3.26403017859e-01 1.23410269913e-01
    CT2 CLA 1 3.71504765465e-01 1.62045623205e-01
    CT2 POT 1 3.26403017859e-01 1.23410269913e-01
   CT2A CLA 1 3.71504765465e-01 1.62045623205e-01
   CT2A POT 1 3.26403017859e-01 1.23410269913e-01
    CT3 CLA 1 3.71504765465e-01 1.62045623205e-01
    CT3 POT 1 3.26403017859e-01 1.23410269913e-01
   CTL1 CLA 1 3.71504765465e-01 1.62045623205e-01
   CTL1 POT 1 3.26403017859e-01 1.23410269913e-01
   CTL2 CLA 1 3.71504765465e-01 1.62045623205e-01
   CTL2 POT 1 3.26403017859e-01 1.23410269913e-01
   CTL3 CLA 1 3.71504765465e-01 1.62045623205e-01
   CTL3 POT 1 3.26403017859e-01 1.23410269913e-01
   CTL5 CLA 1 3.71504765465e-01 1.62045623205e-01
   CTL5 POT 1 3.26403017859e-01 1.23410269913e-01
      N CLA 1 3.67050271874e-01 1.62045623205e-02
      N POT 1 3.21948524268e-01 1.23410269913e-02
    NH1 CLA 1 3.40323310330e-01 7.24690057887e-01
    NH1 POT 1 2.95221562724e-01 5.51907505294e-01
      O CLA 1 3.26959829558e-01 5.61342505072e-01
      O POT 1 2.81858081952e-01 4.27505715330e-01
    OBL CLA 1 3.26959829558e-01 5.61342505072e-01
    OBL POT 1 2.81858081952e-01 4.27505715330e-01

In [ ]:
%%writefile POT.ipt
[ moleculetype ]
POT 1
[ atoms ]
     1 POT 1 POT POT 1 1.000000 39.0983 ; qtot 1.000

%%writefile CLA.ipt
[ moleculetype ]
CLA 1
[ atoms ]
     1 CLA 38 CLA CLA 1 -1.000000 35.4500 ; qtot -1.000

In [ ]:
%%bash
source "/usr/local/gromacs/bin/GMXRC.bash"
cp protein/gromacs/step*_input.gro "conf.gro"
cp -r protein/gromacs/{index.ndx,topol.top,toppar} .
if [[ ! -s "conf.gro" || ! -s "index.ndx" || ! -s "topol.top" ]]; then
  echo "Error: could not extract protein system" >&2
  exit 1
fi
function gro_residues {
  for f in "$@"; do tail -n+3 "${f}" | head -n-1; done | cut -c6-9 | sort | uniq -c | sort -r | awk '{ print $2 }'
}
if fgrep -q "[ MEMB ]" "index.ndx"; then
  gmx editconf -f "conf.gro" -n "index.ndx" -o "MEMB.gro" <<< "MEMB"
  gro_residues "MEMB.gro" > "MEMB.txt"
  rm "MEMB.gro"
fi
gmx editconf -f "conf.gro" -n "index.ndx" -o "SOLV.gro" <<< "SOLV"
gro_residues "SOLV.gro" > "SOLV.txt"
rm "SOLV.gro"
rm "index.ndx"

In [ ]:
%%bash
source "/usr/local/gromacs/bin/GMXRC.bash"
eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
:> "add.txt"
:> "restrain.txt"
for d in ligand_*; do
  IFS=_ read type i <<< $d
  if [[ "${i}" == "*" ]]; then continue; fi
  if [[ -s "${type}_${i}/lig/lig.rtf" && -s "${type}_${i}/lig/lig.prm" ]]; then
    name_="LIG "
    n=$(find -maxdepth 1 -name "${type}_*" -type d | wc -l)
    if (( $n > 1 )); then name_="${name_::$((4 - ${#n}))}${n}"; fi
    name="$(echo $name_)"
    namell="${name,,}"
    mkdir -p "${name}"
    { echo "* For use with CGenFF version 4.6"; cat "${type}_${i}/lig/lig.rtf"; echo "read para"; cat "${type}_${i}/lig/lig.prm"; } | sed "s/ lig / ${name_}/g" > "${name}/${name}.str"
    conda activate "biopython"
    obabel "${type}_${i}/ligandrm.pdb" -O "${name}/${name}.mol2" --title "${name}" --partialcharge none
    pushd "${name}"
    conda activate "charmm2gmx"
    cgenff_charmm2gmx.py "${name}" "${name}.mol2" "${name}.str" "${START}/charmm36.ff"
    popd
    gmx editconf -f "${name}/${namell}_ini.pdb" -o "${name}.gro"
    cp "${name}/${namell}.prm" "toppar/${name}.prm"
    cp "${name}/${namell}.itp" "toppar/${name}.itp"
    if [[ ! -d "toppar/charmm36.ff" ]]; then cp -r "${START}/charmm36.ff" "toppar/"; fi
  elif [[ -s "${type}_${i}/gromacs/topol.top" ]]; then
    name="$(tail -n1 "${type}_${i}/gromacs/topol.top" | awk '{ print $1 }')"
    gmx editconf -f "${type}_${i}/ligandrm.pdb" -o "${name}.gro"
    cp "${type}_${i}/gromacs/charmm36.itp" "toppar/forcefield_${name}.itp"
    cp "${type}_${i}/gromacs/${name}.itp" "toppar/"
  else
    echo "Error: incompatible molecule: ${type}_${i}" >&2
    exit 1
  fi
  echo "${name}" >> "add.txt"
  echo "${name}" >> "restrain.txt"
done

with open("add.txt") as f:
  for l in f:
    l = l.strip()
    x = Itp(f"toppar/{l}.itp")
    atom_id_map = interleave_H_in_topology(x)
    y = Gro(f"{l}.gro")
    rearrange_coordinates(y, atom_id_map)
    with open(f"toppar/{l}.itp", "w") as o: x.print(file=o)
    with open(f"{l}.gro", "w") as o: y.print(file=o)

In [ ]:
%%bash
source "/usr/local/gromacs/bin/GMXRC.bash"
while read -r name; do
  if fgrep -q "POSRES_FC_LIG" "toppar/${name}.itp"; then continue; fi
  gmx genrestr -f "${name}.gro" -fc 9999 <<< "0"
  { echo ""; echo "#ifdef POSRES"; tail -n+3 "posre.itp" | sed "s/9999/POSRES_FC_LIG/g"; echo "#endif"; echo ""; } >> "toppar/${name}.itp"
  rm "posre.itp"
done < "restrain.txt"

if not os.path.isdir("toppar/charmm36.ff"):
  if not os.path.isfile("toppar/forcefield_POT_CLA.itp"):
    !cp "forcefield_POT_CLA.itp" "toppar/forcefield_POT_CLA.itp"
    !echo "POT_CLA" >> "add.txt"
  if not os.path.isfile("toppar/POT.itp"):
    !cp "POT.itp" "toppar/POT.itp"
    !echo "POT" >> "add.txt"
  if not os.path.isfile("toppar/CLA.itp"):
    !cp "CLA.itp" "toppar/CLA.itp"
    !echo "CLA" >> "add.txt"

forcefield_itps = ["toppar/forcefield.itp"]
try:
  with open("add.txt") as f:
    forcefield_itps += [f"toppar/forcefield_{l.strip()}.itp" for l in f if os.path.isfile(f"toppar/forcefield_{l.strip()}.itp")]
except FileNotFoundError: pass

if os.path.isdir("toppar/charmm36.ff"):
  with open("topol.top") as f, open("topol.top.new", "w") as o:
    for l in f:
      if l.rstrip() == '#include "toppar/forcefield.itp"':
        print('#include "toppar/charmm36.ff/forcefield.itp"', file=o)
      else: o.write(l)
  os.rename("topol.top.new", "topol.top")
  for itp in forcefield_itps: os.remove(itp)
elif len(forcefield_itps) > 1:
  combine_itp_files(forcefield_itps, "toppar/forcefield.itp")
  for itp in forcefield_itps[1:]: os.remove(itp)

prm_itp_files = list()
try:
  with open("add.txt") as f:
    prm_itp_files = [f"toppar/{l.strip()}.{ext}" for l in f for ext in ("itp", "prm") if os.path.isfile(f"toppar/{l.strip()}.{ext}")]
except FileNotFoundError: pass
if prm_itp_files:
  prm_files = [f for f in prm_itp_files if f.endswith(".prm")]
  itp_files = [f for f in prm_itp_files if f.endswith(".itp")]
  with open("topol.top") as f, open("topol.top.new", "w") as o:
    l_prev = ""
    for l in f:
      if "forcefield.itp" in l_prev and "forcefield.itp" not in l:
        for prm_file in prm_files: print(f'#include "{prm_file}"', file=o)
      if l_prev.startswith("#include") and not l.startswith("#include"):
        for itp_file in itp_files: print(f'#include "{itp_file}"', file=o)
      o.write(l)
      l_prev = l
  os.rename("topol.top.new", "topol.top")

In [ ]:
%%bash -s "$remove_cmaps_bash"
remove_cmaps_bash="$1"
if $remove_cmaps_bash; then
  pushd "toppar"
  while read -r f; do
    in_cmap=false
    while IFS="" read -r l; do
      if $in_cmap; then
        if [[ "${l}" =~ ^[[:space:]]*\[ ]]; then in_cmap=false; fi
        if ! $in_cmap || [[ "${l}" =~ ^[[:space:]]*$ || "${l}" =~ ^[[:space:]]*'#' || "${l}" =~ ^[[:space:]]*';' ]]; then printf "%s\n" "${l}"; fi
      elif [[ "${l}" =~ ^[[:space:]]*\[[[:space:]]*[cC][mM][aA][pP][[:space:]]*\][[:space:]]*$ ]]; then
        in_cmap=true
      else
        printf "%s\n" "${l}"
      fi
    done < "${f}" > "${f}.tmp"
    mv "${f}.tmp" "${f}"
  done < <(egrep -l -i '^\s*\[\s*cmap\s*\]\s*$' -r .)
  popd
fi

In [ ]:
%%bash
eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
conda activate "biopython"
python3 superpose.py protein/gromacs/step*_input.pdb "protein/step1_pdbreader.pdb"
rms="$(cat "rms.txt")"
if perl -e "exit !(${rms} > 1.0)"; then
  echo "Error: failed to match CHARMM-GUI protein input coordinates to output coordinates: RMSD = ${rms}" >&2
  exit 1
fi

import numpy as np
ro = np.load("ro.npy")
tr = np.load("tr.npy")
try:
  with open("restrain.txt") as f:
    for l in f:
      l = l.strip()
      x = Gro(f"{l}.gro")
      pos = np.array(x.pos)
      new_pos = (pos @ ro) + (tr / 10.)
      x.pos = [tuple(row) for row in new_pos]
      if x.vel is not None:
        vel = np.array(x.vel)
        new_vel = vel @ ro
        x.vel = [tuple(row) for row in new_vel]
      x.box_explicit_diag = False
      x.box = (0., 0., 0.)
      with open(f"{l}.gro", "w") as o: x.print(file=o)
except FileNotFoundError: pass

In [ ]:
%%bash
source "/usr/local/gromacs/bin/GMXRC.bash"
n=$(ls "#topol.top."*"#" 2> /dev/null | wc -l)
cp "topol.top" "#topol.top.${n}#"
while read -r name; do
  echo "0 0 0" > "positions.dat"
  water="$(egrep "(TIP|OPC)" "SOLV.txt" | head -n1)"
  dr=0.0289
  bash insert_molecules_auto_topol "topol.top" "${name}" "${water}" \
    -f "conf.gro" -ci "${name}.gro" -ip "positions.dat" -rot none -dr $dr $dr $dr -replace "resname ${water}" -try 99 -scale 0.235 -o "out.gro" \
  && mv "out.gro" "conf.gro"
done < "restrain.txt"

In [ ]:
%%bash
step0=$(ls -v "protein/gromacs/" | fgrep "minimization.mdp")
cp "protein/gromacs/$step0" "pre_0.mdp"
sed -i"" '/^define/d' "pre_0.mdp"
source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1
gmx grompp -f "pre_0.mdp" -o "pre_0.tpr" -c "conf.gro" -p "topol.top" -maxwarn 999
gmx genion -s "pre_0.tpr" -o "conf.gro" -p "topol.top" -pname "POT" -nname "CLA" -neutral < <(egrep "(TIP|OPC)" "SOLV.txt" | head -n1)
if ! grep -q "POT" "SOLV.txt" && grep -q "^POT " "topol.top"; then echo "POT" >> "SOLV.txt"; fi
if ! grep -q "CLA" "SOLV.txt" && grep -q "^CLA " "topol.top"; then echo "CLA" >> "SOLV.txt"; fi

In [ ]:
%%bash
source "/usr/local/gromacs/bin/GMXRC.bash"
gmx make_ndx -f "conf.gro" < /dev/null 2> /dev/null | fgrep " : " > "index_autodetect.txt"
nr=$(cat "index_autodetect.txt" | wc -l)

SOLV_defn="$(cat "SOLV.txt" | tr "\n" "@" | sed 's/^/"/; s/@$/"/; s/@/" | "/g')"
{
  echo "${SOLV_defn}"
  echo "name ${nr} SOLV"
} > "index_commands.txt"
nr=$(($nr + 1))
if [[ ! -s "MEMB.txt" ]]; then
  { echo '! "SOLV"'; echo "name ${nr} SOLU"; } >> "index_commands.txt"
else
  MEMB_defn="$(cat "MEMB.txt" | tr "\n" "@" | sed 's/^/"/; s/@$/"/; s/@/" | "/g')"
  { echo "${MEMB_defn}"; echo "name ${nr} MEMB"; } >> "index_commands.txt"
  nr=$(($nr + 1))
  { echo '! "SOLV" & ! "MEMB"'; echo "name ${nr} SOLU"; } >> "index_commands.txt"
  nr=$(($nr + 1))
  { echo '"SOLU" | "MEMB"'; echo "name ${nr} SOLU_MEMB"; } >> "index_commands.txt"
fi
echo "q" >> "index_commands.txt"
gmx make_ndx -f "conf.gro" -o "index.ndx" < "index_commands.txt"

In [ ]:
%%bash
step0=$(ls -v "protein/gromacs/" | fgrep "minimization.mdp")
cp "protein/gromacs/$step0" "pre_0.mdp"
i=1
while read -r step; do
  cp "protein/gromacs/$step" "pre_$i.mdp"
  i=$(($i + 1))
done < <(ls -v "protein/gromacs/" | fgrep "equilibration.mdp")

if [[ -s "restrain.txt" ]]; then
  for f in pre_*.mdp; do
    bb="$(head -n1 "$f" | egrep -o "\-DPOSRES_FC_BB=[^ ]+")"
    sed -i"" "s/$bb/$(echo "$bb" | sed 's/_BB/_LIG/') $bb/" "$f"
  done
fi

In [ ]:
%%bash -s "$do_double_eq_bash"
do_double_eq_bash="$1"
if $do_double_eq_bash; then
  for f in pre_*.mdp; do
    pre_nsteps="$(sed -n -E 's/^(\s*nsteps\s*=\s*)([0-9]+)(.*?)$/\2/p' "${f}" | head -n1)"
    double_nsteps="$(perl -l <<< "print(2.0 * ${pre_nsteps})")"
    sed -i"" -E 's/^(\s*nsteps\s*=\s*)([0-9]+)(.*?)$/\1'"${double_nsteps}"'\3/' "${f}"
  done
fi

In [ ]:
%%writefile "production.mdp"
integrator = md
dt = 0.002
nsteps = 500000
comm-mode = Linear
nstxout = 10000
nstvout = 10000
nstenergy = 0
cutoff-scheme = Verlet
nstlist = 100
coulombtype = PME
rcoulomb = 1.2
vdwtype = Cut-off
vdw-modifier = Force-switch
rvdw-switch = 1.0
rvdw = 1.2
tcoupl = V-Rescale
tc-grps = %tc-grps%
tau-t = %tau-t%
ref-t = %ref-t%
pcoupl = C-Rescale
pcoupltype = %pcoupltype%
tau-p = 5.0
compressibility = %compressibility%
ref-p = %ref-p%
constraints = h-bonds
constraint-algorithm = LINCS
continuation = yes

ref_t="$(awk '($1 == "ref_t" || $1 == "ref-t") { print $3; exit }' "pre_1.mdp")"
sed -i"" "/^#@markdown/d" "production.mdp"
if [[ -s "MEMB.txt" ]]; then
  sed -i"" "
    s/%tc-grps%/SOLU SOLV MEMB /g;
    s/%tau-t% /1.0 1.0 1.0 /g;
    s/%ref-t% /${ref_t} ${ref_t} ${ref_t}/g;
    s/%pcoupltype% /semiisotropic /g;
    s/%compressibility%/4.5e-5 4.5e-5/g;
    s/%ref-p% /1.0 1.0 /g;
  " "production.mdp"
else
  sed -i"" "
    s/%tc-grps%/SOLU SOLV /g;
    s/%tau-t% /1.0 1.0 /g;
    s/%ref-t% /${ref_t} ${ref_t}/g;
    s/%pcoupltype% /isotropic/g;
    s/%compressibility%/4.5e-5 /g;
    s/%ref-p% /1.0 /g;
  " "production.mdp"
fi

In [ ]:
%%writefile "run.bash"
output_folder="$1"
cp conf.gro restraint.gro
if ! mkdir -p "${output_folder}"; then
  echo "Error: invalid folder: ${output_folder}" >&2
  exit 1
fi

source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1
echo "Notice: saving output to folder: ${output_folder}"
sleep 1
i=0
while [[ -s "pre_${i}.mdp" ]]; do
  if [[ -s "${output_folder}/pre_${i}.gro" ]]; then
    cp "${output_folder}/pre_${i}.gro" .
  else
    if (( $i == 0 )); then
      gmx grompp -f "pre_${i}.mdp" -o "pre_${i}.tpr" -c "conf.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
      gmx mdrun -deffnm "pre_${i}" || exit $?
    else
      prev=$(($i - 1))
      gmx grompp -f "pre_${i}.mdp" -o "pre_${i}.tpr" -c "pre_${prev}.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
      gmx mdrun -v -stepout 1000 -deffnm "pre_${i}" -bonded gpu || exit $?
    fi
    cp "pre_${i}.gro" "pre_${i}.log" "${output_folder}/"
  fi
  i=$(($i + 1))
done

i=$(($i - 1))
cp "production.mdp" "${output_folder}/grompp.mdp"
cp "pre_${i}.gro" "${output_folder}/conf.gro"
cp -r "index.ndx" "restraint.gro" "topol.top" "toppar" "${output_folder}/"
gmx trjconv -s "pre_${i}.tpr" -f "pre_${i}.gro" -o "conf.pdb" <<< "0"
cp "conf.pdb" "${output_folder}/"
exit 0

!bash "run.bash" "$output_folder"
!sleep 10

In [ ]:
disconnect = True #@param {type: "boolean"}
if disconnect and output_folder.startswith("/content/drive/MyDrive/"):
  from google.colab import drive, runtime
  drive.flush_and_unmount()
  runtime.unassign()